

# Projeto 2 - Ciência dos Dados

Nome: Amanda Rosa do Carmo

Nome: Beatriz Cabral

Nome: Sophia Kerber Shigueoka

Serão permitidos grupos de três pessoas, mas com uma rubrica mais exigente. Grupos deste tamanho precisarão fazer um questionário de avaliação de trabalho em equipe

___
# Classificador automático de sentimento


## Preparando o ambiente no jupyter:

In [1]:
%%capture

#Instalando o tweepy
!pip install tweepy

In [2]:
import tweepy
import math
import os.path
import pandas as pd
import json
from random import shuffle

In [3]:
import re 

def cleanup(text):
    """
        Função de limpeza muito simples que troca alguns sinais básicos por espaços
    """
    punctuation = '[!\-.:?;•,]' # Note que os sinais [] são delimitadores de um conjunto.
    pattern = re.compile(punctuation)
    text_subbed = re.sub(pattern, ' ', text)
    return text_subbed

___
## Autenticando no  Twitter

* Conta: ***@KerberSophie***

In [4]:
#Dados de autenticação do twitter:

#Coloque aqui o identificador da conta no twitter: @KerberSophie

#leitura do arquivo no formato JSON
with open('auth.pass') as fp:    
    data = json.load(fp)

#Configurando a biblioteca. Não modificar
auth = tweepy.OAuthHandler(data['consumer_key'], data['consumer_secret'])
auth.set_access_token(data['access_token'], data['access_token_secret'])

___
## Etapas do projeto:

### Escolha de um produto e coleta das mensagens


Quantidade mínima de mensagens capturadas:
Escolhemos 1480 a partir de um método empírico. Quando testamos com 1000 não conseguimos tweets suficientes, e chegamos
a conclusão de que o problema era proveniente do set(), que diminuía a quantidade de tweets baixados. Assim, decidiu-se
aumentar a quantidade de tweets para 1500, já que com 1000 ficaram falatando 360 tweets. Com 1500, ficamos com 20 tweets a mais.
como eram somente 20 tweets, imaginamos que poucos deles seriam retweets, então tiramos exatamente 20 tweets dos 1500.
Com 1480 tweets, conseguimos 399 tweets de teste, ou seja, faltou somente 1. não achamos que 1 tweets afetará tanto a base
de dados, então decidimos manter os 1480.

In [5]:
#Produto escolhido:
produto = '13 Reasons Why'

n = 1480
#Quantidade mínima de mensagens para a base de treinamento:
t = 600

#Filtro de língua, escolha uma na tabela ISO 639-1.
lang = 'pt'

Capturando os dados do twitter:

In [6]:
#Cria um objeto para a captura
api = tweepy.API(auth, wait_on_rate_limit=True, wait_on_rate_limit_notify=True)

#Inicia a captura, para mais detalhes: ver a documentação do tweepy
i = 1
msgs = []
for msg in tweepy.Cursor(api.search, q=produto, lang=lang).items():    
    msgs.append(msg.text.lower())
    i += 1
    if i > n:
        break

#Embaralhando as mensagens para reduzir um possível viés
list = []
shuffle(msgs)

#Usando o set() para tirar os retweets
listaset = set(msgs)

#Adicionando o resultado do set() a uma lista para evitar o erro TypeError: 'set' object is not subscriptable
for value in listaset:
    list.append(value)

Salvando os dados em uma planilha Excel:

In [7]:
#Linhas abaixo foram comentadas uma vez que o conjunto de dados era o mesmo, mas o número de tweets foi alterado.

# #Verifica se o arquivo não existe para não substituir um conjunto pronto
# if not os.path.isfile('./{0}.xlsx'.format(produto)):
    
    #Abre o arquivo para escrita
writer = pd.ExcelWriter('{0} novo.xlsx'.format(produto))

    #divide o conjunto de mensagens em duas planilhas
dft = pd.DataFrame({'Treinamento' : pd.Series(list[:t])})
dft.to_excel(excel_writer = writer, sheet_name = 'Treinamento', index = False)

dfc = pd.DataFrame({'Teste' : pd.Series(list[t:])})
dfc.to_excel(excel_writer = writer, sheet_name = 'Teste', index = False)

    #fecha o arquivo
writer.save()

___
### Montando o Classificador Naive-Bayes

Considerando apenas as mensagens da planilha Treinamento, ensine  seu classificador.

In [8]:
import tweepy
import math
import os.path
import pandas as pd
import json
from random import shuffle

In [9]:
import re 

def cleanup(text):
    """
        Função de limpeza muito simples que troca alguns sinais básicos por espaços
    """
    punctuation = '[!\-.:?;•,]' # Note que os sinais [] são delimitadores de um conjunto.
    pattern = re.compile(punctuation)
    text_subbed = re.sub(pattern, ' ', text)
    return text_subbed

In [10]:
excel = pd.read_excel('13 Reasons Why v2.xlsx', sheet_name='Treinamento')
excel.head()

,Treinamento,Categoria
0,alguém aq ja assistiu a 3° temp de 13 reasons ...,O
1,(na 3° temporada de 13 reasons why) eu estou t...,R
2,13 ep de 13 reasons why p tentar fazer vc sent...,N
3,nossa nem sabia que tinha lançado a 3° tempora...,O
4,estava começando a ver o primeiro episódio de...,N


In [11]:
excel_treino = pd.read_excel('13 Reasons Why v2.xlsx', sheet_name='Teste')
excel_treino.rename(columns={'Unnamed: 1':'Clas_manual'}, inplace=True)
excel_treino.head()

,Teste,Clas_manual
0,a coisa mais idiota de 13 reasons why foi colo...,N
1,atribuí nota 8 ao episódio 3x1 - yeah. i'm the...,O
2,rt @citou13reasons: ''quando você julga uma pe...,O
3,"terminei 13 reasons why também, meu hobbie é c...",R
4,rt @wivisampaio: saiu a nova temporada de elit...,O


___
### Função para limpar e criar tabela de frequências relativas:

In [12]:
def FreqRel(cat):
    texto_completo = ' '.join(excel[excel.Categoria==cat].Treinamento)
    semhttp = re.sub(r'http\S+', '', texto_completo)
    text_limpo = cleanup(semhttp)
    rel = text_limpo.split()
    freq_rel = pd.Series(rel).value_counts(True)
    return freq_rel

FreqRel('P')

reasons           0.062772
why               0.059285
13                0.059285
de                0.054054
a                 0.033130
temporada         0.025283
que               0.017437
o                 0.012206
eu                0.011334
é                 0.011334
foi               0.010462
e                 0.009590
da                0.008718
série             0.008718
mas               0.007847
q                 0.007847
uma               0.007847
essa              0.007847
tá                0.007847
terceira          0.006975
muito             0.006975
boa               0.006975
não               0.006975
foda              0.006103
atribuí           0.006103
nota              0.006103
ao                0.006103
episódio          0.006103
#bancodeseries    0.006103
temp              0.005231
                    ...   
bullying          0.000872
deus              0.000872
importa           0.000872
realidade         0.000872
saco              0.000872
confusa           0.000872
p

In [13]:
# # Estas funções dividem a função anterior
def Limpa(texto):
    semhttp = re.sub(r'http\S+', '', texto)
    text_limpo = cleanup(semhttp)
    rel = text_limpo.split()
    return(rel)

def FreqRel(Limpa):
    freq_rel = pd.Series(Limpa).value_counts(True)
    return freq_rel


___
### Descobrindo P(N), P(R), P(P) e P(0)

In [14]:
texto_1 = ' '.join(excel[excel.Categoria=='O'].Treinamento)
texto_2 = ' '.join(excel[excel.Categoria=='R'].Treinamento)
texto_3 = ' '.join(excel[excel.Categoria=='P'].Treinamento)
texto_4 = ' '.join(excel[excel.Categoria=='N'].Treinamento)

texto_1 = Limpa(texto_1)
texto_2 = Limpa(texto_2)
texto_3 = Limpa(texto_3)
texto_4 = Limpa(texto_4)

#juntando todos os textos
texto_enorme = texto_1 + texto_2 + texto_3 + texto_4
set(texto_enorme)
dic_global = 0
for palavra in texto_enorme:
        dic_global+=1
    
print('Todas as palavras somadas:')
print(dic_global)

def prob_cat(cat):
    palavras = 0
    texto_completo = ' '.join(excel[excel.Categoria==cat].Treinamento)
    texto_completo.replace('•', '')
    semhttp = re.sub(r'http\S+', '', texto_completo)
    text_limpo = cleanup(semhttp)
    rel = text_limpo.split()
    set(rel)
    # Para verificar se o código era coerente, imprimimos o total de palavras de cada categoria
    for palavra in rel:
        palavras += 1
    prob_final = palavras/dic_global
    return prob_final

Todas as palavras somadas:
10095


In [15]:
print("Probabilidade da Categoria 'Neutro'")
print(prob_cat('O'))
print('')
print("Probabilidade da Categoria 'Negativo'")
print(prob_cat('N'))
print('')
print("Probabilidade da Categoria 'Positivo'")
print(prob_cat('P'))
print('')
print("Probabilidade da Categoria 'Reação'")
print(prob_cat('R'))



print('')
print("Probabilidade Total (se tudo der certo, deve ser 1)")
print(prob_cat('O') + prob_cat('N') + prob_cat('P') + prob_cat('R'))

Probabilidade da Categoria 'Neutro'
0.5090638930163447

Probabilidade da Categoria 'Negativo'
0.2220901436354631

Probabilidade da Categoria 'Positivo'
0.11362060425953442

Probabilidade da Categoria 'Reação'
0.15522535908865776

Probabilidade Total (se tudo der certo, deve ser 1)
1.0


In [16]:
try:
    Fqr = FreqRel('R')
    Fqn = FreqRel('N')
    Fqp = FreqRel('P')
    Fqo = FreqRel('O')
except: 
    pass

for palavra, frequencia in Fqr.items():
    print(palavra)

R


In [17]:
Neutro_set = set(FreqRel(0).index)
Reacao_set = set(FreqRel('R').index)

___
### Verificando a performance

Agora você deve testar o seu classificador com a base de Testes.

###   P(N | palavras) = P(N) * P(cada_palavra | N)

NÃO É PRECISO DIVIDIR PELA PROBABILIDADE 

In [18]:
# probP = 0
# probN = 0
# probR = 0
# probO = 0
# i = 0
# dic_catR = {}
# dic_catP = {}
# dic_catN = {}
# dic_catO = {}


# #Para cada tweet na planilha:
# for tweet in excel_treino['Teste']:
#     tweet = cleanup(tweet.lower())
#     pal_tweet = tweet.split()
#     #Para cada palavra de cada tweet:
#     for palavra in pal_tweet:
#         um = True
#         for palavra1, frequencia in Fqr.items():
#             if um == True:
#                 #Seta a frquencia inicial de cada palavra para 1
#                 dic_catR[tweet] = 1
#             um = False
#             # probabilidade total são todas as probabilidades das palavras multiplicadas pela probabilidade de certa categoria
#             dic_catR[tweet] = dic_catR[tweet] *  prob_cat('R') * frequencia
                              
#         for palavra1, frequencia in Fqn.items():
#             if um == True:
#                 dic_catN[tweet] = 1
#             um = False
#             dic_catN[tweet] = dic_catN[tweet] *  prob_cat('N') * frequencia
            
#         for palavra1, frequencia in Fqp.items():
#             if um == True:
#                 dic_catP[tweet] = 1
#             um = False
#             dic_catP[tweet] = dic_catP[tweet] *  prob_cat('P') * frequencia

#         for palavra1, frequencia in Fqo.items():
#             if um == True:
#                 dic_catP[tweet] = 1
#             um = False
#             dic_catO[tweet] = dic_catP[tweet] *  prob_cat('O') * frequencia     

#         #Não precisa dividir pela probabilidade do tweet pois cancelaria.
    
# # fazer contador pra ver qual é a maior probabilidade, e adicionar em uma nova coluna do dataframe
# #  DataFrame.insert(2, 'Predição', [lista com letra de cada linha], True)

In [19]:
prob_R = 1
prob_tweet = 1
dicR = {}
for tweet in excel_treino['Teste']:
    tweet = cleanup(tweet)
    pal_tweet = tweet.split()
    for palavra_teste in pal_tweet:
        for palavra, freq in Fqr.items():
            if palavra_teste in Fqr.items():
                probabilidade = freq
            else:
                probabilidade = 1
            prob_tweet = prob_tweet * freq
    prob_tweet = prob_tweet * prob_cat('R')
    dicR[tweet] = prob_tweet

In [20]:
prob_N = 1
prob_tweet = 1
dicN = {}
for tweet in excel_treino['Teste']:
    tweet = cleanup(tweet)
    pal_tweet = tweet.split()
    for palavra_teste in pal_tweet:
        for palavra, freq in Fqn.items():
            if palavra_teste in Fqn.items():
                probabilidade = freq
            else:
                probabilidade = 1
            prob_tweet = prob_tweet * freq
    prob_tweet = prob_tweet * prob_cat('N')
    dicN[tweet] = prob_tweet

In [21]:
prob_P = 1
prob_tweet = 1
dicP = {}
for tweet in excel_treino['Teste']:
    tweet = cleanup(tweet)
    pal_tweet = tweet.split()
    for palavra_teste in pal_tweet:
        for palavra, freq in Fqp.items():
            if palavra_teste in Fqp.items():
                probabilidade = freq
            else:
                probabilidade = 1
            prob_tweet = prob_tweet * freq
    prob_tweet = prob_tweet * prob_cat('P')
    dicP[tweet] = prob_tweet

In [22]:
prob_O = 1
prob_tweet = 1
dicO = {}
for tweet in excel_treino['Teste']:
    tweet = cleanup(tweet)
    pal_tweet = tweet.split()
    for palavra_teste in pal_tweet:
        for palavra, freq in Fqo.items():
            if palavra_teste in Fqo.items():
                probabilidade = freq
            else:
                probabilidade = 1
            prob_tweet = prob_tweet * freq
    prob_tweet = prob_tweet * prob_cat('O')
    dicO[tweet] = math.log(prob_tweet,10)

In [23]:
print(dicN['a coisa mais idiota de 13 reasons why foi colocar o zac e a cheryl nas fitas  botaram eles num nível de fdp igual aos outros desnecessário'])

0.2220901436354631


In [24]:
print(dicO['a coisa mais idiota de 13 reasons why foi colocar o zac e a cheryl nas fitas  botaram eles num nível de fdp igual aos outros desnecessário'])

-0.29322770559448513


In [25]:
print(dicR['a coisa mais idiota de 13 reasons why foi colocar o zac e a cheryl nas fitas  botaram eles num nível de fdp igual aos outros desnecessário'])

0.15522535908865776


In [26]:
print(dicP['a coisa mais idiota de 13 reasons why foi colocar o zac e a cheryl nas fitas  botaram eles num nível de fdp igual aos outros desnecessário'])

0.11362060425953442


In [27]:
tweets_classificados = []

for tweet in excel_treino['Teste']:
    tweet = cleanup(tweet)
    pal_tweet = tweet.split()
    maior = dicN[tweet]
    if maior < dicP[tweet]:
        maior = dicP
        tweets_classificados.append('P')
    elif maior < dicO[tweet]:
        maior = dicO
        tweets_classificados.append('O') 
    elif maior < dicR[tweet]:
        maior = dicR
        tweets_classificados.append('R')
    else:
        tweets_classificados.append('N')
        
    print(dicR[tweet],dicP[tweet],dicO[tweet], dicN[tweet])

0.15522535908865776 0.11362060425953442 -0.29322770559448513 0.2220901436354631
0.024094912104202743 0.012909641712301731 -0.5864554111889704 0.049324031900020625
0.0037401413835845172 0.0014668012921258133 -0.8796831167834557 0.010954381329355743
0.000580564789309256 0.00016665884914000078 -1.1729108223779405 0.0024328601228742522
9.01183778947602e-05 1.8935879144485476e-05 -1.466138527972426 0.0005403142541341331
1.3988657569201508e-05 2.1515060305819554e-06 -1.759366233566911 0.00011999847030893772
2.1713943943475743e-06 2.444554152627541e-07 -2.052593939161396 2.6650477506947832e-05
3.3705547458570075e-07 2.7775171996669537e-08 -2.345821644755881 5.918808377471722e-06
5.2319557075363356e-08 3.155831825674092e-09 -2.6390493503503665 1.314509002703477e-06
8.121322034382802e-09 3.5856751897456e-10 -2.9322770559448514 2.9193949322052457e-07
1.260635129061699e-09 4.0740658173731586e-11 -3.225504761539337 6.48368839822106e-08
1.9568254058837865e-10 4.628978199630523e-12 -3.51873246713382

4.322717019240462e-266 1.548556430166e-310 -96.17868743499113 4.5878105231260223e-215
6.70995301550253e-267 1.7594791732545e-311 -96.47191514058562 1.0189074980533474e-215
1.0415548662994022e-267 1.99913086847e-312 -96.7651428461801 2.2628931259391825e-216
1.616757281318636e-268 2.2714245727e-313 -97.05837055177459 5.025662593715351e-217
2.5096172955188734e-269 2.580806325e-314 -97.35159825736908 1.1161501273016163e-217
3.895562458720232e-270 2.93232774e-315 -97.64482596296355 2.4788594209115637e-218
6.046900815071426e-271 3.3317285e-316 -97.93805366855806 5.5053024484237005e-219
9.386323503929593e-272 3.78553e-317 -98.23128137415253 1.2226734115270862e-219
1.4569954364197794e-272 4.301143e-318 -98.52450907974702 2.7154371358531224e-220
2.2616263980879587e-273 4.887e-319 -98.81773678534151 6.030718235346904e-221
3.5106176976759103e-274 5.553e-320 -99.11096449093598 1.3393630791132003e-221
5.449368927447401e-275 6.31e-321 -99.40419219653047 2.9745933862028675e-222
8.458802485695967e-276

### **DASHBOARD**

In [28]:
print(len(tweets_classificados))

398


In [29]:
# from time import sleep
# import pandas as pd

# for e in range(3):
#     !jupyter nbconvert --to notebook --execute Projeto2_layout.ipynb
#     dashboard = pd.readExcel("13 Reasons Why novo.xlsx")
#     print(f"dashoard: {dashoard}")
#     sleep(5)

excel_treino['Categorias'] = tweets_classificados
excel_treino.to_excel('fill_auto1.xlsx')

___
### Concluindo

## Aperfeiçoamento:

Os trabalhos vão evoluir em conceito dependendo da quantidade de itens avançados:

* Limpar: \n, :, ", ', (, ), etc SEM remover emojis
* Corrigir separação de espaços entre palavras e emojis ou emojis e emojis
* Propor outras limpezas e transformações que não afetem a qualidade da informação ou classificação
* Criar categorias intermediárias de relevância baseadas na probabilidade: ex.: muito relevante, relevante, neutro, irrelevante, muito irrelevante (3 categorias: C, mais categorias conta para B)
* Explicar por que não posso usar o próprio classificador para gerar mais amostras de treinamento
* Propor diferentes cenários para Naïve Bayes fora do contexto do projeto
* Sugerir e explicar melhorias reais com indicações concretas de como implementar (indicar como fazer e indicar material de pesquisa)
* Montar um dashboard que periodicamente realiza análise de sentimento e visualiza estes dados

___
### Por que não se pode utilizar o próprio classificador para gerar mais amostras de treinamento?

    Ao se utilizar o próprio classificador para gerar mais amostras de treinamento, o classificador acaba ficando 
    'viciado', ou seja, acaba-se por prejudicar os resultados obtidos, e eles acabam não tendo uma qualidade boa.
    Seria o contrário de puxar mais tweets, que melhora a qualidade dos dados e dos resultados, por ter um espaço 
    amostral muito maior.

### Outras limpezas e tranformações:
___
    - Remover URLs
    - Trocar vírgulas por espaços
    - 
    
    

### Outros cenários de Naive Bayes fora do contexto do projeto
___

    - Previsão de Chuva
    - Filtros de e-mail (Spam)

# Referências

[Naive Bayes and Text Classification](https://arxiv.org/pdf/1410.5329.pdf)  **Mais completo**

[A practical explanation of a Naive Bayes Classifier](https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/) **Mais simples**

https://towardsdatascience.com/naive-bayes-classifier-81d512f50a7c

https://www.geeksforgeeks.org/naive-bayes-classifiers/

https://www.analyticsvidhya.com/blog/2017/09/naive-bayes-explained/